## Import Libraries

In [2]:
import numpy as np
import pandas as pd
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, NonNegativeReals, SolverFactory

## Directory

## Risk Parity

In [3]:
def risk_parity_pyomo(returns, asset_classes, asset_class_constraints):
    """
    Computes portfolio weights using Risk Parity with Pyomo, allowing user-defined asset allocation constraints.
    
    Parameters:
        returns (pd.DataFrame): Historical returns of assets.
        asset_classes (dict): Dictionary mapping asset names to their asset class ('stocks', 'bonds', 'deposits').
        asset_class_constraints (dict): User-defined allocation constraints for asset classes, 
                                        e.g., {'stocks': (0.3, 0.7), 'bonds': (0.2, 0.5), 'deposits': (0.1, 0.3)}.

    Returns:
        dict: Optimized portfolio weights.
    """
    # Compute covariance matrix
    Sigma = returns.cov().values
    assets = list(returns.columns)
    num_assets = len(assets)

    # Create Pyomo Model
    model = ConcreteModel()
    
    # Variables: Portfolio Weights (No short-selling allowed)
    model.w = Var(range(num_assets), domain=NonNegativeReals)
    
    # Constraint: Fully Invested Portfolio (sum of weights = 1)
    model.full_investment = Constraint(expr=sum(model.w[i] for i in range(num_assets)) == 1)

    # Portfolio Variance Calculation
    def portfolio_variance(model):
        return sum(model.w[i] * Sigma[i, j] * model.w[j] for i in range(num_assets) for j in range(num_assets))

    # Risk Contribution Calculation
    def risk_contribution(i):
        return model.w[i] * sum(Sigma[i, j] * model.w[j] for j in range(num_assets)) / portfolio_variance(model)

    # Risk Parity Objective: Minimize the standard deviation of risk contributions
    def objective_function(model):
        risk_contributions = [risk_contribution(i) for i in range(num_assets)]
        mean_risk = sum(risk_contributions) / num_assets
        return sum((risk_contributions[i] - mean_risk) ** 2 for i in range(num_assets))
    
    model.objective = Objective(rule=objective_function, sense=1)  # Minimize risk contribution disparity

    # Apply User-Defined Asset Class Constraints
    for asset_class, (lower, upper) in asset_class_constraints.items():
        indices = [i for i, asset in enumerate(assets) if asset_classes[asset] == asset_class]
        if indices:
            model.add_component(f"{asset_class}_min", Constraint(expr=sum(model.w[i] for i in indices) >= lower))
            model.add_component(f"{asset_class}_max", Constraint(expr=sum(model.w[i] for i in indices) <= upper))

    # Solve the Optimization Problem
    solver = SolverFactory('ipopt')
    solver.solve(model, tee=True)

    # Extract Optimized Weights
    weights = {assets[i]: model.w[i].value for i in range(num_assets)}
    return weights


In [ ]:
np.random.seed(42)
returns_data = pd.DataFrame(np.random.randn(100, 5) / 100, columns=['AAPL', 'GOOGL', 'TLT', 'IEF', 'CASH'])

# Define asset classes
asset_classes = {
    'AAPL': 'stocks', 
    'GOOGL': 'stocks', 
    'TLT': 'bonds', 
    'IEF': 'bonds', 
    'CASH': 'deposits'
}

# User-Defined Allocation Constraints
asset_class_constraints = {
    'stocks': (0.3, 0.7),   # Stocks allocation between 30% and 70%
    'bonds': (0.2, 0.5),    # Bonds allocation between 20% and 50%
    'deposits': (0.1, 0.3)  # Deposits allocation between 10% and 30%
}

# Compute Optimized Weights
optimal_weights = risk_parity_pyomo(returns_data, asset_classes, asset_class_constraints)
print("\nOptimized Portfolio Weights:", optimal_weights)